<div style='text-align: center;'>
    <img src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQzJzIHdangJTrH2mFXFgsLjuLCjpfXXwbxg&usqp=CAU' width='100'/>
    <h1>Sharif University of Technology</h1>
    <h2>Natural Language Processing</h2>
    <h3>Final Project</h3>
    <h4>Spoiler classification and summary generation</h4>
    <p><strong>Authors:</strong> Abolfazl eshagh</p>
    <p><strong></strong></p>
</div>
<hr/>


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd MyDrive

/content/drive/MyDrive


In [4]:
mkdir NLP_project

In [5]:
ls

'02 Greatest.mp3'
"02 - I'll be waiting.mp3"
'03 - Chasing pavements.mp3'
'03 Lucky You (feat. Joyner Lucas).mp3'
'09 Kamikaze.mp3'
'11 Nice Guy (feat. Jessie Reyez).mp3'
'11 - Right as rain.mp3'
'12 Good Guy (feat. Jessie Reyez).mp3'
'13 - Melt my heart to stone.mp3'
'14 - One and only.mp3'
'15 - Cold shoulder.mp3'
'16 - He wont go.mp3'
'17 - My same.mp3'
'18 - Dont you remember.mp3'
'19 - If it hadn'\''t been for love ( live at the royal albert hall ).mp3'
 1nmj.zip
 2.pdf
'35- حسابان - پاسخ تابع 2.pdf'
'35-حسابان - سوال تابع 1.pdf'
'35- حسابان - سوال تابع 2.pdf'
'35- حسابان - مثلثات 1.pdf'
'35- زبان - درس 121.pdf'
'35- شیمی - سوال فصل 121.pdf'
'35-عربی - درس 121.pdf'
'35- گسسته - نظریه اعداد.pdf'
'36- حسابان - سوال حد.pdf'
'36-فیزیک -سوال نوسان و موج.pdf'
 3D_pointcloud@
 4_5983170123735766507.xlsx
 747898arabi_jamee_test_olgoo_darse1.pdf
 97C03T38.pdf
 97C11T16.pdf
 97C11T17.pdf
 97C11T17P.pdf
 97C11T18.pdf
 97C13T32.pdf
 98.03.20.pdf
 98C03O02.pdf
 98C11T27.pdf
 98C11T27P.pdf
 98C

In [6]:
cd NLP_project

/content/drive/MyDrive/NLP_project


In [7]:
ls

In [8]:
!kaggle datasets download -d rmisra/imdb-spoiler-dataset

Dataset URL: https://www.kaggle.com/datasets/rmisra/imdb-spoiler-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
100% 331M/331M [00:04<00:00, 71.1MB/s]
100% 331M/331M [00:04<00:00, 81.5MB/s]


In [9]:
ls

imdb-spoiler-dataset.zip


In [11]:
!unzip imdb-spoiler-dataset.zip

Archive:  imdb-spoiler-dataset.zip
  inflating: IMDB_movie_details.json  
  inflating: IMDB_reviews.json       


In [12]:
ls

IMDB_movie_details.json  IMDB_reviews.json  imdb-spoiler-dataset.zip


In [14]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 req

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Trainer, TrainingArguments
import gc
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

In [4]:
cd drive/MyDrive/NLP_project

/content/drive/MyDrive/NLP_project


In [5]:
ls

IMDB_movie_details.json  IMDB_reviews.json  imdb-spoiler-dataset.zip


In [6]:
movie_details = pd.read_json('IMDB_movie_details.json', lines=True)
reviews = pd.read_json('IMDB_reviews.json', lines=True)

In [7]:
movie_details = movie_details[movie_details['plot_synopsis'] != ""]
movie_details

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...
5,tt0286716,"Bruce Banner, a brilliant scientist with a clo...",2h 18min,"[Action, Sci-Fi]",5.7,2003-06-20,Bruce Banner (Eric Bana) is a research scienti...
...,...,...,...,...,...,...,...
1563,tt0120655,An abortion clinic worker with a special herit...,2h 10min,"[Adventure, Comedy, Drama]",7.3,1999-11-12,The film opens with a homeless man (Bud Cort) ...
1565,tt0276751,Twelve year old Marcus Brewer lives with his c...,1h 41min,"[Comedy, Drama, Romance]",7.1,2002-05-17,Will Freeman (Hugh Grant) is a 38-year-old bac...
1567,tt0289879,Evan Treborn grows up in a small town with his...,1h 53min,"[Sci-Fi, Thriller]",7.7,2004-01-23,"In the year 1998, Evan Treborn (Ashton Kutcher..."
1568,tt1723811,Brandon is a 30-something man living in New Yo...,1h 41min,[Drama],7.2,2012-01-13,"Brandon (Michael Fassbender) is a successful, ..."


In [8]:
reviews = reviews[reviews['review_summary'] != '']
reviews

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
573908,8 August 1999,tt0139239,ur0100166,False,"Go is wise, fast and pure entertainment. Assem...",10,The best teen movie of the nineties
573909,31 July 1999,tt0139239,ur0021767,False,"Well, what shall I say. this one´s fun at any ...",9,Go - see the movie
573910,20 July 1999,tt0139239,ur0392750,False,"Go is the best movie I have ever seen, and I'v...",10,It's the best movie I've ever seen
573911,11 June 1999,tt0139239,ur0349105,False,Call this 1999 teenage version of Pulp Fiction...,3,Haven't we seen this before?


In [11]:
review1 = reviews.loc[1]
plot1 = movie_details.loc[1]
print(review1)
print()
print(plot1)

review_date                                        6 September 2000
movie_id                                                  tt0111161
user_id                                                   ur0842118
is_spoiler                                                     True
review_text       The Shawshank Redemption is without a doubt on...
rating                                                           10
review_summary           Simply amazing. The best film of the 90's.
Name: 1, dtype: object

movie_id                                                 tt1204975
plot_summary     Billy (Michael Douglas), Paddy (Robert De Niro...
duration                                                  1h 45min
genre                                                     [Comedy]
rating                                                         6.6
release_date                                            2013-11-01
plot_synopsis    Four boys around the age of 10 are friends in ...
Name: 1, dtype: object


In [12]:
movie_details = movie_details.drop(columns = ['movie_id', 'duration', 'genre', 'rating', 'release_date'])
reviews = reviews.drop(columns = ['review_date', 'movie_id', 'user_id', 'is_spoiler', 'rating'])
movie_details

,plot_summary,plot_synopsis
0,"Former CIA analyst, Jack Ryan is in England wi...","Jack Ryan (Ford) is on a ""working vacation"" in..."
1,"Billy (Michael Douglas), Paddy (Robert De Niro...",Four boys around the age of 10 are friends in ...
3,"Fred C. Dobbs and Bob Curtin, both down on the...",Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,Tracy Flick is running unopposed for this year...,Jim McAllister (Matthew Broderick) is a much-a...
5,"Bruce Banner, a brilliant scientist with a clo...",Bruce Banner (Eric Bana) is a research scienti...
...,...,...
1563,An abortion clinic worker with a special herit...,The film opens with a homeless man (Bud Cort) ...
1565,Twelve year old Marcus Brewer lives with his c...,Will Freeman (Hugh Grant) is a 38-year-old bac...
1567,Evan Treborn grows up in a small town with his...,"In the year 1998, Evan Treborn (Ashton Kutcher..."
1568,Brandon is a 30-something man living in New Yo...,"Brandon (Michael Fassbender) is a successful, ..."


In [13]:
reviews

,review_text,review_summary
0,"In its Oscar year, Shawshank Redemption (writt...",A classic piece of unforgettable film-making.
1,The Shawshank Redemption is without a doubt on...,Simply amazing. The best film of the 90's.
2,I believe that this film is the best story eve...,The best story ever told on film
3,"**Yes, there are SPOILERS here**This film has ...",Busy dying or busy living?
4,At the heart of this extraordinary movie is a ...,"Great story, wondrously told and acted"
...,...,...
573908,"Go is wise, fast and pure entertainment. Assem...",The best teen movie of the nineties
573909,"Well, what shall I say. this one´s fun at any ...",Go - see the movie
573910,"Go is the best movie I have ever seen, and I'v...",It's the best movie I've ever seen
573911,Call this 1999 teenage version of Pulp Fiction...,Haven't we seen this before?


In [17]:
from sklearn.model_selection import train_test_split
plot_dataset = movie_details[['plot_summary', 'plot_synopsis']]

X = plot_dataset['plot_synopsis']
y = plot_dataset['plot_summary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

train_set = pd.DataFrame({'plot_synopsis': X_train, 'plot_summary': y_train})
test_set = pd.DataFrame({'plot_synopsis': X_test, 'plot_summary': y_test})
val_set = pd.DataFrame({'plot_synopsis': X_val, 'plot_summary': y_val})


In [18]:
len(train_set), len(test_set), len(val_set)

(937, 201, 201)

In [20]:
train_set.loc[166]['plot_synopsis']

'It\'s 1995. Jimmy "B-Rabbit" Smith Jr. (Eminem) is a young and frustrated blue-collar factory worker. He has moved back north of 8 Mile to the run-down trailer home in Warren, belonging to his alcoholic mother, Stephanie (Kim Basinger), his younger sister, Lily (Chloe Greenfield) and Stephanie\'s abusive live-in boyfriend, Greg (Michael Shannon).Jimmy is focused on getting his rap career started. He seems to have stalled in progress; choking in the middle of a rap battle and getting booed off stage. Prior to the events of the film, Jimmy had broken up with his pregnant girlfriend, Janeane (Taryn Manning). During the film, Jimmy begins a new relationship with Alex (Brittany Murphy), who he meets at the factory he works in.Jimmy begins to realise that his life has remained pretty much the same since he graduated from high school.At first, he considers himself to be a victim of circumstances and blames others for his problems. Over time, Jimmy begins to take responsibility for his life a

In [21]:
train_set.loc[166]['plot_summary']

"This is the inspiring captivating story of the legendary rapper Eminem. The troubled young aspiring rapper from a ghetto in Michigan must exert his last chances to become successful while dealing with his life in ruins. All is seemingly lost. He is now single, has only a few friends, an insane/alcoholic mother, and is dealt with poverty and living in a violent city on 8 mile. His only way out of the ghetto and torturous life he's living in is with his talent in rapping. Will B-Rabbit prevail and seize the shot he's given or will he let it slip?                Written by\nojmessier23"

In [16]:
# Load BART tokenizer and model
model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [22]:
# Tokenize data
train_encodings = tokenizer(train_set['plot_synopsis'].tolist(), truncation=True, padding="max_length", max_length=512, return_tensors="pt")
train_labels = tokenizer(train_set['plot_summary'].tolist(), truncation=True, padding="max_length", max_length=128, return_tensors="pt")
val_encodings = tokenizer(val_set['plot_synopsis'].tolist(), truncation=True, padding="max_length", max_length=512, return_tensors="pt")
val_labels = tokenizer(val_set['plot_summary'].tolist(), truncation=True, padding="max_length", max_length=128, return_tensors="pt")
test_encodings = tokenizer(test_set['plot_synopsis'].tolist(), truncation=True, padding="max_length", max_length=512, return_tensors="pt")
test_labels = tokenizer(test_set['plot_summary'].tolist(), truncation=True, padding="max_length", max_length=128, return_tensors="pt")

In [23]:
# Create dataset class
class PlotDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels['input_ids'][idx].clone().detach()
        return item

In [24]:
train_dataset_prepared = PlotDataset(train_encodings, train_labels)
val_dataset = PlotDataset(val_encodings, val_labels)
test_dataset = PlotDataset(test_encodings, test_labels)

In [27]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_prepared,
    eval_dataset=val_dataset,
)

In [29]:
trainer.train()

Step,Training Loss,Validation Loss
500,2.728100,2.586693


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Step,Training Loss,Validation Loss
500,2.728100,2.586693
1000,1.969300,2.710294
1500,1.494200,2.824270
2000,1.405600,2.972204
2500,0.807000,3.267314
3000,0.606000,3.473132
3500,0.581100,3.602122
4000,0.423200,3.724041
4500,0.295700,3.774722


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=4690, training_loss=1.3974467617108115, metrics={'train_runtime': 3814.8153, 'train_samples_per_second': 2.456, 'train_steps_per_second': 1.229, 'total_flos': 1.015288506089472e+16, 'train_loss': 1.3974467617108115, 'epoch': 10.0})

In [30]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [31]:
# Save model and tokenizer
# model.save_pretrained('./saved_model')
# tokenizer.save_pretrained('./saved_model')
torch.save(model.state_dict, 'bart_plot_summary_generation10.pth')

In [32]:
# Evaluate model
# model = MBartForConditionalGeneration.from_pretrained('./saved_model')
# tokenizer = MBart50TokenizerFast.from_pretrained('./saved_model')
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# val_dataset = PlotDataset(test_encodings, test_labels)
test_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

all_predictions = []
all_references = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model.generate(input_ids, max_length=64, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
        all_predictions.extend(outputs)
        all_references.extend(labels)

In [33]:
# Decode predictions and references
decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in all_predictions]
decoded_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in all_references]

In [35]:
!pip install rouge_score
!pip install bert_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=dff0fa73859ac2f140725dd8b1a24a2460a265ff2fb77fccc92c587f18413b3b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  

In [36]:
# Compute metrics
from datasets import load_metric
# rouge_types=["rouge1", "rouge2", "rougeL"]
rouge = load_metric("rouge")
# rouge = rouge(rouge_types)
bleu = load_metric("bleu")
from bert_score import score



<ipython-input-36-0e0b19336a83>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


The repository for bleu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bleu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

+-----------------------------+--------+
|            Metric           | Value  |
+-----------------------------+--------+
|          BLEU Score         | 0.0163 |
|       Brevity Penalty       | 0.4838 |
|         Length Ratio        | 0.5794 |
|      Translation Length     |  9799  |
|       Reference Length      | 16913  |
|      Precisions 1-gram      | 0.3294 |
|      Precisions 2-gram      | 0.0559 |
|      Precisions 3-gram      | 0.0135 |
|      Precisions 4-gram      | 0.0051 |
|   ROUGE-rouge1 (Precision)  | 0.3795 |
|    ROUGE-rouge1 (Recall)    | 0.2223 |
|   ROUGE-rouge1 (F-Measure)  | 0.2744 |
|   ROUGE-rouge2 (Precision)  | 0.0785 |
|    ROUGE-rouge2 (Recall)    | 0.0444 |
|   ROUGE-rouge2 (F-Measure)  | 0.0555 |
|   ROUGE-rougeL (Precision)  | 0.2330 |
|    ROUGE-rougeL (Recall)    | 0.1368 |
|   ROUGE-rougeL (F-Measure)  | 0.1685 |
| ROUGE-rougeLsum (Precision) | 0.2333 |
|   ROUGE-rougeLsum (Recall)  | 0.1370 |
| ROUGE-rougeLsum (F-Measure) | 0.1686 |
|     BERTScore 

In [37]:
rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
bleu_result = bleu.compute(predictions=[pred.split() for pred in decoded_preds], references=[[ref.split()] for ref in decoded_labels])
P, R, F1 = score(decoded_preds, decoded_labels, lang='multilingual')
bertscore_result = {"precision": P.mean().item(), "recall": R.mean().item(), "f1": F1.mean().item()}

results = {"bleu": bleu_result, "rouge": rouge_result, "bertscore": bertscore_result}

# Print results
from prettytable import PrettyTable

def print_results(results):
    table = PrettyTable()
    table.field_names = ["Metric", "Value"]

    bleu_scores = results['bleu']
    table.add_row(["BLEU Score", f"{bleu_scores['bleu']:.4f}"])
    table.add_row(["Brevity Penalty", f"{bleu_scores['brevity_penalty']:.4f}"])
    table.add_row(["Length Ratio", f"{bleu_scores['length_ratio']:.4f}"])
    table.add_row(["Translation Length", bleu_scores['translation_length']])
    table.add_row(["Reference Length", bleu_scores['reference_length']])
    for i, precision in enumerate(bleu_scores['precisions'], 1):
        table.add_row([f"Precisions {i}-gram", f"{precision:.4f}"])

    rouge_scores = results['rouge']
    for key in rouge_scores:
        score = rouge_scores[key].mid
        table.add_row([f"ROUGE-{key} (Precision)", f"{score.precision:.4f}"])
        table.add_row([f"ROUGE-{key} (Recall)", f"{score.recall:.4f}"])
        table.add_row([f"ROUGE-{key} (F-Measure)", f"{score.fmeasure:.4f}"])

    bert_scores = results['bertscore']
    table.add_row(["BERTScore Precision", f"{bert_scores['precision']:.4f}"])
    table.add_row(["BERTScore Recall", f"{bert_scores['recall']:.4f}"])
    table.add_row(["BERTScore F1", f"{bert_scores['f1']:.4f}"])

    print(table)

print_results(results)


+-----------------------------+--------+
|            Metric           | Value  |
+-----------------------------+--------+
|          BLEU Score         | 0.0163 |
|       Brevity Penalty       | 0.4838 |
|         Length Ratio        | 0.5794 |
|      Translation Length     |  9799  |
|       Reference Length      | 16913  |
|      Precisions 1-gram      | 0.3294 |
|      Precisions 2-gram      | 0.0559 |
|      Precisions 3-gram      | 0.0135 |
|      Precisions 4-gram      | 0.0051 |
|   ROUGE-rouge1 (Precision)  | 0.3795 |
|    ROUGE-rouge1 (Recall)    | 0.2223 |
|   ROUGE-rouge1 (F-Measure)  | 0.2744 |
|   ROUGE-rouge2 (Precision)  | 0.0785 |
|    ROUGE-rouge2 (Recall)    | 0.0444 |
|   ROUGE-rouge2 (F-Measure)  | 0.0555 |
|   ROUGE-rougeL (Precision)  | 0.2330 |
|    ROUGE-rougeL (Recall)    | 0.1368 |
|   ROUGE-rougeL (F-Measure)  | 0.1685 |
| ROUGE-rougeLsum (Precision) | 0.2333 |
|   ROUGE-rougeLsum (Recall)  | 0.1370 |
| ROUGE-rougeLsum (F-Measure) | 0.1686 |
|     BERTScore 

In [42]:
test_set

,plot_synopsis,plot_summary
266,"In the opening shot, an alien spaceship flies ...","A US research station, Antarctica, early-winte..."
1245,"Alex Browning is aordinary teenager, who has t...",Alex is boarding his plane to France on a scho...
1425,"Sandra Bullock plays Lucy Ellenore Moderatz, a...",Lucy's life consists of constant loneliness th...
190,It is 1936 and a money runner is carrying the ...,"Johnny Hooker, a small time grifter, unknowing..."
656,Milo (Sran Todorovi) is a semi-retired Serbian...,"In Serbia, the retired porn star Milos is marr..."
...,...,...
1491,"Chaplin is the story of Charles Chaplin, the f...","The biography of Charles Chaplin, filmmaker ex..."
226,The film begins with Bradley Fine (Jude Law) i...,"Susan Cooper is an unassuming, deskbound CIA a..."
1510,The movie takes place in Los Angeles in the ye...,Straight from the pages of a pulp comic from a...
574,"Using a tapa cloth to animate her story, Gramm...",Moana Waialiki is a sea voyaging enthusiast an...


In [43]:
input_plot = test_set.loc[266]
input_plot['plot_summary']

'A US research station, Antarctica, early-winter 1982. The base is suddenly buzzed by a helicopter from the nearby Norwegian research station. They are trying to kill a dog that has escaped from their base. After the destruction of the Norwegian chopper the members of the US team fly to the Norwegian base, only to discover them all dead or missing. They do find the remains of a strange creature the Norwegians burned. The Americans take it to their base and deduce that it is an alien life form. After a while it is apparent that the alien can take over and assimilate into other life forms, including humans, and can spread like a virus. This means that anyone at the base could be inhabited by The Thing, and tensions escalate.                Written by\ngrantss'

In [44]:
input_plot['plot_synopsis']

'In the opening shot, an alien spaceship flies through space and enters Earth\'s atmosphere near Antarctica. Whether or not the ship crashes or lands on Earth is unknown.In Antarctica, the winter months of 1982, a helicopter flies hurriedly after a husky. The man riding next to the pilot shoots a rifle and drops grenades hoping to kill the dog. The runs toward an American research base, Outpost 31, where a 12-man research crew is getting ready for the upcoming winter. The helicopter, which they see is from a Norwegian research base, flies in and lands. The Americans watch puzzled as two men that emerge from the helicopter speak frantically in their native language. One of them pulls the pin on a grenade but clumsily throws it behind him. As he frantically tries to find it in the snow, it explodes, killing him and taking the helicopter with it. The pilot (named Matias in the prequel) is shooting his rifle hysterically and talking frantically; no one can understand what he is saying and 

In [45]:

# Example article for prediction
example_article = input_plot['plot_synopsis']
input_encoding = tokenizer(example_article, return_tensors="pt", max_length=256, padding="max_length", truncation=True)

print("input plot:")
print(example_article)

print()

# Generate title
model.eval()
with torch.no_grad():
    input_encoding = {k: v.to(device) for k, v in input_encoding.items()}
    outputs = model.generate(input_encoding['input_ids'], max_length=64, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    predicted_title = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Predicted plot summary:")
print(predicted_title)

input plot:
In the opening shot, an alien spaceship flies through space and enters Earth's atmosphere near Antarctica. Whether or not the ship crashes or lands on Earth is unknown.In Antarctica, the winter months of 1982, a helicopter flies hurriedly after a husky. The man riding next to the pilot shoots a rifle and drops grenades hoping to kill the dog. The runs toward an American research base, Outpost 31, where a 12-man research crew is getting ready for the upcoming winter. The helicopter, which they see is from a Norwegian research base, flies in and lands. The Americans watch puzzled as two men that emerge from the helicopter speak frantically in their native language. One of them pulls the pin on a grenade but clumsily throws it behind him. As he frantically tries to find it in the snow, it explodes, killing him and taking the helicopter with it. The pilot (named Matias in the prequel) is shooting his rifle hysterically and talking frantically; no one can understand what he is s

# **Review summary generation**